In [1]:
import tensorflow as tf
import src.gan as gan

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
nn = gan.GAN(12, 16, 4)

x = tf.placeholder(shape=[None, 32, 32, 1], dtype=tf.float32)
x_fake, p_real, p_fake = nn(x)

gen_loss, discrim_loss = nn.make_losses(x)

In [4]:
train_summaries = [
    tf.summary.scalar('train/loss/gen', gen_loss),
    tf.summary.scalar('train/loss/discrim', discrim_loss),
    tf.summary.image('train/gen', x_fake),
]
test_summaries = [
    tf.summary.scalar('test/loss/gen', gen_loss),
    tf.summary.scalar('test/loss/discrim', discrim_loss),
    tf.summary.image('test/gen', x_fake),
]

train_merged = tf.summary.merge(train_summaries)
test_merged = tf.summary.merge(test_summaries)

In [5]:
gen_opt = tf.train.MomentumOptimizer(0.01, momentum=0.9)
gnvs = gen_opt.compute_gradients(gen_loss, var_list=nn.generator.variables)
gen_step = gen_opt.apply_gradients(tf.contrib.training.clip_gradient_norms(gnvs, 1))

discrim_opt = tf.train.AdamOptimizer()
discrim_step = discrim_opt.minimize(discrim_loss, var_list=nn.discriminator.variables)

train_step = tf.group([gen_step, discrim_step])

saver = tf.train.Saver()

In [6]:
logdir = '/tmp/gan/0'
!rm -rf {logdir}
epochs = 50

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logdir, sess.graph)
    
    for i in range(epochs*1000):
        batch_x, _ = mnist.train.next_batch(50)
        _, train_summ = sess.run([train_step, train_merged], 
                                 feed_dict={x: gan.GAN.preprocess(batch_x)})
        
        if i % 10 == 0:
            writer.add_summary(train_summ, i)
        
        if i % 100 == 0:
            L, test_summ = sess.run([gen_loss, test_merged], 
                                                feed_dict={x: 
                        gan.GAN.preprocess(mnist.test.images[:100, ...])})
            print('\rStep: {} Loss: {}'.format(i, L), end='', flush=True)
            writer.add_summary(test_summ, i)
    save_path = saver.save(sess, logdir+"/gan.ckpt")

Step: 4900 Loss: 2.8724722862243652

KeyboardInterrupt: 